In [1]:
import xarray as xr
import pandas as pd
import plotly.express as px

In [2]:
hot_ctd = xr.load_dataset("../../2_processed/hot_ctd.nc")
hot_primary_production = xr.load_dataset("../../2_processed/hot_primary_production.nc")
hot_zooplankton = xr.load_dataset("../../2_processed/hot_zooplankton.nc")

In [3]:
hot_zooplankton

<xarray.Dataset> Size: 2MB
Dimensions:  (time: 548, is_day: 2, lat: 1, lon: 1, depth: 8, frac: 5)
Coordinates:
  * time     (time) datetime64[ns] 4kB 1994-02-17 1994-02-18 ... 2022-09-02
  * is_day   (is_day) bool 2B False True
  * lat      (lat) float64 8B 22.75
  * lon      (lon) int64 8B -158
  * depth    (depth) int64 64B 50 100 150 200 250 300 350 400
  * frac     (frac) float64 40B 0.2 0.5 1.0 2.0 5.0
Data variables:
    vol      (time, is_day, lat, lon, depth, frac) float64 351kB nan nan ... nan
    svol     (time, is_day, lat, lon, depth, frac) float64 351kB nan nan ... nan
    wwt      (time, is_day, lat, lon, depth, frac) float64 351kB nan nan ... nan
    dwt      (time, is_day, lat, lon, depth, frac) float64 351kB nan nan ... nan
    carb     (time, is_day, lat, lon, depth, frac) float64 351kB nan nan ... nan
    nit      (time, is_day, lat, lon, depth, frac) float64 351kB nan nan ... nan
    abnd     (time, is_day, lat, lon, depth, frac) float64 351kB nan nan ... nan

## Manage index

---


In [4]:
times = pd.DataFrame(
    {
        "btl_time": hot_ctd.time.to_series(),
        "zpk_time": hot_zooplankton.time.to_series(),
        "pp_time": hot_primary_production.time.to_series(),
    }
)
fig = px.histogram(
    times,
    x=["btl_time", "zpk_time", "pp_time"],
    title="time distribution",
    opacity=0.5,
    log_y=True,
    nbins=30,
    barmode="overlay",
    marginal="box",
    labels={"value": "time (m)", "variable": "Datasets"},
)

fig.show()

In [5]:
depths = pd.DataFrame(
    {
        "ctd_depth": hot_ctd.where(hot_ctd.depth < 400, drop=True).depth.to_series(),
        "zpk_depth": hot_zooplankton.depth.to_series(),
        "pp_depth": hot_primary_production.depth.to_series(),
    }
)
px.box(
    depths.stack().reset_index().rename(columns={"level_1": "dataset"}),
    x="dataset",
    y="depth",
)

## Gathering

---


In [6]:
hot_zooplankton = hot_zooplankton.rename({"nit": "nitrogen"})
hot_ctd = hot_ctd.rename({"nit": "nitrate"})

In [7]:
final_dataset = xr.merge([hot_zooplankton, hot_primary_production, hot_ctd])
final_dataset = final_dataset.rename(
    {"lat": "latitude", "lon": "longitude", "frac": "sieve_size"}
)
final_dataset.load()

<xarray.Dataset> Size: 4MB
Dimensions:     (time: 687, is_day: 2, latitude: 1, longitude: 1, depth: 8,
                 sieve_size: 5)
Coordinates:
  * time        (time) datetime64[ns] 5kB 1988-10-31 1988-12-02 ... 2022-09-02
  * is_day      (is_day) bool 2B False True
  * latitude    (latitude) float64 8B 22.75
  * longitude   (longitude) int64 8B -158
  * depth       (depth) int64 64B 50 100 150 200 250 300 350 400
  * sieve_size  (sieve_size) float64 40B 0.2 0.5 1.0 2.0 5.0
Data variables: (12/25)
    vol         (time, is_day, latitude, longitude, depth, sieve_size) float64 440kB ...
    svol        (time, is_day, latitude, longitude, depth, sieve_size) float64 440kB ...
    wwt         (time, is_day, latitude, longitude, depth, sieve_size) float64 440kB ...
    dwt         (time, is_day, latitude, longitude, depth, sieve_size) float64 440kB ...
    carb        (time, is_day, latitude, longitude, depth, sieve_size) float64 440kB ...
    nitrogen    (time, is_day, latitude, longitude, depth, sieve_size) float64 440kB ...
    ...          ...
    temp        (time, latitude, longitude, depth) float64 44kB 26.21 ... nan
    sal         (time, latitude, longitude, depth) float64 44kB 35.21 ... nan
    oxy         (time, latitude, longitude, depth) float64 44kB 217.5 ... nan
    xmiss       (time, latitude, longitude, depth) float64 44kB 3.42 ... nan
    fluor       (time, latitude, longitude, depth) float64 44kB 0.1021 ... nan
    nitrate     (time, latitude, longitude, depth) float64 44kB nan nan ... nan

In [8]:
final_dataset.to_netcdf("../../3_post_processed/hot_product.nc", mode="w")